In [1]:
import sys
import os

# Get the current working directory (which is the directory where the notebook is running)
current_dir = os.getcwd()

# Get the parent directory
parent_dir = os.path.abspath(os.path.join(current_dir, ".."))

# Set pwd one directory back
os.chdir(parent_dir)

In [2]:
%load_ext autoreload
%autoreload 2

import os
from os.path import join, exists
import logging
import json
import shutil

from water_rights_visualizer import water_rights_visualizer
from water_rights_visualizer.S3_source import S3Source

In [3]:
logger = logging.getLogger(__name__)

In [4]:
import pymongo


def build_mongo_client_and_collection():
    # todo: read from ENV vars and then use defaults if not available
    user = os.environ.get("MONGO_INITDB_ROOT_USERNAME", "")
    cred = os.environ.get("MONGO_INITDB_ROOT_PASSWORD", "")
    # host = os.environ.get("MONGO_HOST", "water-rights-visualizer-mongo")
    host = os.environ.get("MONGO_HOST", "localhost")
    port = os.environ.get("MONGO_PORT", 27017)
    if isinstance(port, str) and port.isdigit():
        port = int(port)

    database = os.environ.get("MONGO_DATABASE", "water")
    collection = os.environ.get("MONGO_COLLECTION", "report_queue")

    mongo_str = "mongodb://{}:{}@{}:{}".format(user, cred, host, port)

    client = pymongo.MongoClient(host=host, username=user, password=cred, port=port, directConnection=True)

    db = client[database]
    collect = db[collection]

    return collect


report_queue = build_mongo_client_and_collection()

In [17]:
%load_ext autoreload
%autoreload 2

config_filepath = os.path.expanduser("~/data/water_rights_runs/03479_2023_2023_1755108445534/config.json")
config = json.load(open(config_filepath, "r"))

temporary_directory = os.path.expanduser("~/data/temp")

name = config["name"]
logger.info(f"name: {name}")
start_year = int(config["start_year"])
logger.info(f"start year: {start_year}")
end_year = int(config["end_year"])
logger.info(f"end year: {end_year}")
working_directory = config["working_directory"]
logger.info(f"working directory: {working_directory}")
geojson_filename = config["geojson_filename"]
logger.info(f"GeoJSON file: {geojson_filename}")
status_filename = config["status_filename"]
logger.info(f"status file: {status_filename}")

start_year = 1999
end_year = 1999

temporary_directory = join(working_directory, "temp")
output_directory = join(working_directory, "output")

input_datastore = S3Source(
    bucket_name="ose-dev-inputs",
    region_name="us-west-2",
    temporary_directory=temporary_directory,
    remove_temporary_files=False,
    aws_profile="ose-nmw",
)

# Uncomment to remove the output on each run
if exists(output_directory):
    shutil.rmtree(output_directory)

record = report_queue.find_one({"key": config["key"]}) or {}
water_rights_visualizer(
    boundary_filename=geojson_filename,
    input_datastore=input_datastore,
    output_directory=output_directory,
    start_year=start_year,
    end_year=end_year,
    requestor=record.get("user", {"sub": "", "name": "", "email": ""}),
    use_stack=True,
)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
[2025-09-12 11:56:05 INFO] name: 03479
[2025-09-12 11:56:05 INFO] start year: 2023
[2025-09-12 11:56:05 INFO] end year: 2023
[2025-09-12 11:56:05 INFO] working directory: /Users/rstonebr/data/water_rights_runs/03479_2023_2023_1755108445534
[2025-09-12 11:56:05 INFO] GeoJSON file: /Users/rstonebr/data/water_rights_runs/03479_2023_2023_1755108445534/03479.geojson
[2025-09-12 11:56:05 INFO] status file: /Users/rstonebr/data/water_rights_runs/03479_2023_2023_1755108445534/status.txt
[2025-09-12 11:56:05 INFO] S3 temporary directory: /Users/rstonebr/data/water_rights_runs/03479_2023_2023_1755108445534/temp
[2025-09-12 11:56:05 INFO] Found credentials in shared credentials file: ~/.aws/credentials
[2025-09-12 11:56:05 INFO] boundary file: /Users/rstonebr/data/water_rights_runs/03479_2023_2023_1755108445534/03479.geojson
[2025-09-12 11:56:05 INFO] output directory: /Users/rstonebr/data/water_rights_runs/03

/opt/homebrew/anaconda3/envs/nmw-runner/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/homebrew/anaconda3/envs/nmw-runner/lib/python3.10/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


[2025-09-12 11:58:04 INFO] Writing cloud coverage to cache: /Users/rstonebr/data/water_rights_runs/03479_2023_2023_1755108445534/output/nan_subsets/03479/cloud_coverage_cache/cloud_coverage_1999_09.json
[2025-09-12 11:58:04 INFO] Year: 1999, Month: 09, Mean Cloud Coverage: 41.27%, Time: 6.67 seconds
[2025-09-12 11:58:04 INFO] Generating figure

[2025-09-12 11:58:04 INFO] Created 1 records
[2025-09-12 11:58:04 INFO] generating figure for year 1999 ROI 03479 units: <water_rights_visualizer.plotting_helpers.MetricETUnit object at 0x337517c40>
[2025-09-12 11:58:04 INFO] rendering month: 1 sub-figure: 0
[2025-09-12 11:58:04 INFO] sub-figure title: January
[2025-09-12 11:58:04 INFO] rendering month: 2 sub-figure: 1
[2025-09-12 11:58:04 INFO] sub-figure title: February
[2025-09-12 11:58:04 INFO] rendering month: 3 sub-figure: 2
[2025-09-12 11:58:04 INFO] sub-figure title: March
[2025-09-12 11:58:04 INFO] rendering month: 4 sub-figure: 3
[2025-09-12 11:58:04 INFO] sub-figure title: April
[2025

/Users/rstonebr/Documents/Programming/Water-Rights-Visualizer/water_rights_visualizer/generate_figure.py:505: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()


[2025-09-12 11:58:04 INFO] finished display_image_tk
[2025-09-12 11:58:04 INFO] finished generating figure
[2025-09-12 11:58:04 INFO] generating figure for year 1999 ROI 03479 units: <water_rights_visualizer.plotting_helpers.ImperialETUnit object at 0x3375143a0>
[2025-09-12 11:58:04 INFO] rendering month: 1 sub-figure: 0
[2025-09-12 11:58:04 INFO] sub-figure title: January
[2025-09-12 11:58:04 INFO] rendering month: 2 sub-figure: 1
[2025-09-12 11:58:04 INFO] sub-figure title: February
[2025-09-12 11:58:04 INFO] rendering month: 3 sub-figure: 2
[2025-09-12 11:58:04 INFO] sub-figure title: March
[2025-09-12 11:58:04 INFO] rendering month: 4 sub-figure: 3
[2025-09-12 11:58:04 INFO] sub-figure title: April
[2025-09-12 11:58:04 INFO] rendering month: 5 sub-figure: 4
[2025-09-12 11:58:04 INFO] sub-figure title: May
[2025-09-12 11:58:04 INFO] rendering month: 6 sub-figure: 5
[2025-09-12 11:58:04 INFO] sub-figure title: June
[2025-09-12 11:58:04 INFO] rendering month: 7 sub-figure: 6
[2025-09-

...

[2025-09-12 11:58:12 INFO] finished generating summary figure
[2025-09-12 11:58:12 INFO] generating summary figure for ROI 03479 units: <water_rights_visualizer.plotting_helpers.AcreFeetETUnit object at 0x36ab19660>
[2025-09-12 11:58:12 INFO] Correcting ETo based on ET for years after 1985
[2025-09-12 11:58:12 INFO] Saving summary figure to /Users/rstonebr/data/water_rights_runs/03479_2023_2023_1755108445534/output/figures/03479/summary_03479_AF.png



/Users/rstonebr/Documents/Programming/Water-Rights-Visualizer/water_rights_visualizer/summary_figure_generator.py:420: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()


[2025-09-12 11:58:12 INFO] finished generating summary figure


/Users/rstonebr/Documents/Programming/Water-Rights-Visualizer/water_rights_visualizer/pdf_report_generator.py:69: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  pdf.savefig(fig, bbox_inches="tight", pad_inches=0)
/Users/rstonebr/Documents/Programming/Water-Rights-Visualizer/water_rights_visualizer/pdf_report_generator.py:84: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  pdf.savefig(fig, bbox_inches="tight", pad_inches=0)


[2025-09-12 11:58:13 INFO] metric report saved to /Users/rstonebr/data/water_rights_runs/03479_2023_2023_1755108445534/output/figures/03479/03479_Report.pdf

[2025-09-12 11:58:13 INFO] imperial report saved to /Users/rstonebr/data/water_rights_runs/03479_2023_2023_1755108445534/output/figures/03479/03479_Imperial_Report.pdf

[2025-09-12 11:58:14 INFO] acre-feet report saved to /Users/rstonebr/data/water_rights_runs/03479_2023_2023_1755108445534/output/figures/03479/03479_AF_Report.pdf

